<a href="https://colab.research.google.com/github/jrg94/CSE5522/blob/lab1/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.0**: Set up the environment (you can click on the play button below to import the appropriate modules).

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

**1.1** Read the data from GitHub into a pandas dataframe. 

In [0]:
micro_rna_data_url = 'https://raw.githubusercontent.com/efosler/cse5522data/master/miRScore-x.csv'
micro_ran_dataframe = pd.read_csv(micro_rna_data_url)
micro_rna_well_data_url = 'https://raw.githubusercontent.com/efosler/cse5522data/master/mRNA-y-well-explained.csv'
micro_rna_well_dataframe = pd.read_csv(micro_rna_well_data_url)